---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Detección de Personas con armas cortas
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Diciembre, 25 de 2022"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Modelo para la creación conjunto de datos con super pixeles
* **Problema:** 
Existen diferentes conjuntos de datos disponibles sin embargo se hace uso del conjunto de datos que corresponde al grupo procesado de personas con armas cortas.

* **Solucion :** 
Se utilizara el trabajo realizado en https://github.com/idealo/image-super-resolution#additional-information, con redes GAN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/personas-armadas-detectadas.tar.gz

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [ ]:
!git clone https://github.com/idealo/image-super-resolution
%cd image-super-resolution
!python setup.py install

Cloning into 'image-super-resolution'...
remote: Enumerating objects: 1203, done.
remote: Total 1203 (delta 0), reused 0 (delta 0), pack-reused 1203
Receiving objects: 100% (1203/1203), 14.98 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (583/583), done.
/content/image-super-resolution
/usr/local/lib/python3.8/dist-packages/setuptools/dist.py:697: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  warnings.warn(
running install
running bdist_egg
running egg_info
creating ISR.egg-info
writing ISR.egg-info/PKG-INFO
writing dependency_links to ISR.egg-info/dependency_links.txt
writing requirements to ISR.egg-info/requires.txt
writing top-level names to ISR.egg-info/top_level.txt
writing manifest file 'ISR.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'ISR.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
run

## Importar Librerias



In [ ]:
import os
import glob
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from PIL import Image
from ISR.models import RDN, RRDN
import shutil

## Constantes

In [ ]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
SAVED_MODEL_PATH_RRDN = r'rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5'
TEST_MODE = False
MAX_IMAGES = 10
mainPath = "./"

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

## Clases

In [ ]:
class SuperResolutionRRDN:
  def __init__(self,sourcePath,outputPath) -> None:
    # model = RDN(weights='noise-cancel')
    self.model = RRDN(weights='gans')
    #rdn = RDN(arch_params={'C': 6, 'D': 20, 'G': 64, 'G0': 64, 'x': 2})
    #rdn.model.load_weights(SAVED_MODEL_PATH_RRDN)
    #self.model = RDN(weights='psnr-small')
    # model = RDN(weights='psnr-large')
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    
  def preprocess_image(self,image_path):
    """ Loads image from path and preprocesses to make it model ready
        Args:
          image_path: Path to the image file
    """
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    # If PNG, remove the alpha channel. The model only supports
    # images with 3 color channels.
    if hr_image.shape[-1] == 4:
      hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)



  def save_image(self, image, filename):
    """
      Saves unscaled Tensor Images.
      Args:
        image: 3D image tensor. [height, width, channels]
        filename: Name of the file to save.
    """
    if not isinstance(image, Image.Image):
      image = tf.clip_by_value(image, 0, 255)
      image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    image.save(filename)
    print("Saved as %s" % filename)

  def improveImage(self,imageName):    
    start = time.time()
    hr_image = Image.open(os.path.join(self.sourcePath,imageName))    
    fake_image = self.model.predict(np.array(hr_image))
    print("Time Taken: %f" % (time.time() - start))       
    self.save_image(tf.squeeze(fake_image), filename=os.path.join(self.outputPath,imageName))

  def improveImageResized(self,hr_image,imageName):    
    start = time.time()     
    fake_image = self.model.predict(np.array(hr_image))
    print("Time Taken: %f" % (time.time() - start))       
    self.save_image(tf.squeeze(fake_image), filename=os.path.join(self.outputPath,imageName))

  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      im = Image.open(imagePath)
      width, height = im.size
      sizeImg = 400, 400
    
      if width>=800 or height>=800:
        print("Large Image Name: %s" % imageName)       
        shutil.copy(imagePath, os.path.join(self.outputPath,imageName))
        print("File copied successfully.")
      else:        
        print("Image Name: %s" % imageName)
        self.improveImage(imageName)  
  
  def plot_image(self, image, title=""):
    """
      Plots images from image tensors.
      Args:
        image: 3D image tensor. [height, width, channels].
        title: Title to display in the plot.
    """
    image = np.asarray(image)
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    plt.imshow(image)
    plt.axis("off")
    plt.title(title)

## Decarga conjunto de datos

In [ ]:
!wget https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/personas-armadas-detectadas.tar.gz -O personas-armadas-detectadas.tar.gz

--2022-12-26 07:12:43--  https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/personas-armadas-detectadas.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73725823 (70M) [application/octet-stream]
Saving to: ‘personas-armadas-detectadas.tar.gz’

personas-armadas-de 100%[===================>]  70.31M   185MB/s    in 0.4s    

2022-12-26 07:12:48 (185 MB/s) - ‘personas-armadas-detectadas.tar.gz’ saved [73725823/73725823]



In [ ]:
!tar -xvf personas-armadas-detectadas.tar.gz

./personas-armadas-detectadas/
./personas-armadas-detectadas/gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
./personas-armadas-detectadas/mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
./personas-armadas-detectadas/gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
./personas-armadas-detectadas/406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.jpg
./personas-armadas-detectadas/armas-403-_jpg.rf.88ae1f16a3e58580396a5d31d4d77210.jpg
./personas-armadas-detectadas/images-of-pistol-in-hand_24_jpeg_jpg.rf.263936679204d70ccd00e13f3111be23.jpg
./personas-armadas-detectadas/mgd_custom_frame1737_jpg.rf.6e82171efd22b487c6eb01cdfa401d23.jpg
./personas-armadas-detectadas/mgd_custom_frame164_jpg.rf.13ae32f9111f7b87ddc76398eb6b945c.jpg
./personas-armadas-detectadas/gmd_4_frame67_jpg.rf.ad437c5e6ffe0fc9fd99db751f722290.jpg
./personas-armadas-detectadas/gmd_4_frame47_jpg.rf.8a10f48f58900ab92d3d55bfb63c08dc.jpg
./personas-armadas-detectadas/images-of-

## Creacion proyecto esrgan

In [ ]:
!rm -rf super-resolution-esrgan-tf2
!mkdir super-resolution-esrgan-tf2
!mkdir "super-resolution-esrgan-tf2/person"

## Creacion Pipeline Super Resolucion RRDN

In [ ]:
sourcePath = os.path.join(mainPath,"personas-armadas-detectadas")
outputPath = os.path.join(mainPath,"super-resolution-esrgan-tf2/person/")
superResolutionRRDN = SuperResolutionRRDN(sourcePath,outputPath)
superResolutionRRDN.processImages()

17462488/17462488 [==============================] - 3s 0us/step
Image Name: gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
1/1 [==============================] - 9s 9s/step
Time Taken: 8.888336
Saved as ./super-resolution-esrgan-tf2/person/gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
Image Name: mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
1/1 [==============================] - 2s 2s/step
Time Taken: 2.065646
Saved as ./super-resolution-esrgan-tf2/person/mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
Image Name: gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
1/1 [==============================] - 1s 677ms/step
Time Taken: 0.734234
Saved as ./super-resolution-esrgan-tf2/person/gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
Large Image Name: 406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.jpg
File copied successfully.
Image Name: armas-403-_jpg.rf.88ae1f16a3e58580396a5d31d4d77210.jpg


## Compresion Archivos

In [ ]:
!rm -rf super-resolution-esrgan-personas-armadas.tar.gz
!tar -czvf super-resolution-esrgan-personas-armadas.tar.gz "./super-resolution-esrgan-tf2/person"

./super-resolution-esrgan-tf2/person/
./super-resolution-esrgan-tf2/person/gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
./super-resolution-esrgan-tf2/person/mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
./super-resolution-esrgan-tf2/person/gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
./super-resolution-esrgan-tf2/person/406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.jpg
./super-resolution-esrgan-tf2/person/armas-403-_jpg.rf.88ae1f16a3e58580396a5d31d4d77210.jpg
./super-resolution-esrgan-tf2/person/images-of-pistol-in-hand_24_jpeg_jpg.rf.263936679204d70ccd00e13f3111be23.jpg
./super-resolution-esrgan-tf2/person/mgd_custom_frame1737_jpg.rf.6e82171efd22b487c6eb01cdfa401d23.jpg
./super-resolution-esrgan-tf2/person/mgd_custom_frame164_jpg.rf.13ae32f9111f7b87ddc76398eb6b945c.jpg
./super-resolution-esrgan-tf2/person/gmd_4_frame67_jpg.rf.ad437c5e6ffe0fc9fd99db751f722290.jpg
./super-resolution-esrgan-tf2/person/gmd_4_frame47_jpg.rf.

## Super Resolicon RDN

## Creacion proyecto

In [ ]:
!rm -rf super-resolution-rdn
!mkdir super-resolution-rdn
!mkdir "super-resolution-rdn/person"

## Clases

In [ ]:
class SuperResolutionRDN:
  def __init__(self,sourcePath,outputPath) -> None:
    # model = RDN(weights='noise-cancel')
    #self.model = RRDN(weights='gans')
    #rdn = RDN(arch_params={'C': 6, 'D': 20, 'G': 64, 'G0': 64, 'x': 2})
    #rdn.model.load_weights(SAVED_MODEL_PATH_RRDN)
    self.model = RDN(weights='psnr-small')
    #self.model = RDN(weights='psnr-large')
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    
  def preprocess_image(self,image_path):
    """ Loads image from path and preprocesses to make it model ready
        Args:
          image_path: Path to the image file
    """
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    # If PNG, remove the alpha channel. The model only supports
    # images with 3 color channels.
    if hr_image.shape[-1] == 4:
      hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

  def save_image(self, image, filename):
    """
      Saves unscaled Tensor Images.
      Args:
        image: 3D image tensor. [height, width, channels]
        filename: Name of the file to save.
    """
    if not isinstance(image, Image.Image):
      image = tf.clip_by_value(image, 0, 255)
      image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    image.save(filename)
    print("Saved as %s" % filename)

  def improveImage(self,imageName):    
    start = time.time()
    hr_image = Image.open(os.path.join(self.sourcePath,imageName))    
    fake_image = self.model.predict(np.array(hr_image))
    print("Time Taken: %f" % (time.time() - start))       
    self.save_image(tf.squeeze(fake_image), filename=os.path.join(self.outputPath,imageName))

  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      im = Image.open(imagePath)
      width, height = im.size
      sizeImg = 400, 400
      if width>=800 or height>=800:
        print("Large Image Name: %s" % imageName)       
        shutil.copy(imagePath, os.path.join(self.outputPath,imageName))
        print("File copied successfully.")
      else:        
        print("Image Name: %s" % imageName)
        self.improveImage(imageName)  
    
  
  def plot_image(self, image, title=""):
    """
      Plots images from image tensors.
      Args:
        image: 3D image tensor. [height, width, channels].
        title: Title to display in the plot.
    """
    image = np.asarray(image)
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    plt.imshow(image)
    plt.axis("off")
    plt.title(title)

## Creacion Pipeline Super Resolucion RDN

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
sourcePath = os.path.join(mainPath,"personas-armadas-detectadas")
outputPath = os.path.join(mainPath,"super-resolution-rdn/person/")
superResolutionRDN = SuperResolutionRDN(sourcePath,outputPath)
superResolutionRDN.processImages()

10694096/10694096 [==============================] - 2s 0us/step
Image Name: gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
1/1 [==============================] - 1s 1s/step
Time Taken: 1.501320
Saved as ./super-resolution-rdn/person/gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
Image Name: mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
1/1 [==============================] - 1s 1s/step
Time Taken: 1.037262
Saved as ./super-resolution-rdn/person/mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
Image Name: gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
1/1 [==============================] - 1s 833ms/step
Time Taken: 0.870425
Saved as ./super-resolution-rdn/person/gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
Large Image Name: 406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.jpg
File copied successfully.
Image Name: armas-403-_jpg.rf.88ae1f16a3e58580396a5d31d4d77210.jpg
1/1 [================

In [ ]:
!rm -rf super-resolution-rdn.tar.gz
!tar -czvf super-resolution-rdn.tar.gz "./super-resolution-rdn/person"

./super-resolution-rdn/person/
./super-resolution-rdn/person/gmd_1_frame143_jpg.rf.d5bd8826c34fab72906ddb22829457de.jpg
./super-resolution-rdn/person/mgd_custom_frame1207_jpg.rf.4a549e41e1649360a04cbf61d1d62bbf.jpg
./super-resolution-rdn/person/gmd_3_frame43_jpg.rf.45a3e2691757b9ed5fd87bf9b7ea47a5.jpg
./super-resolution-rdn/person/406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.jpg
./super-resolution-rdn/person/armas-403-_jpg.rf.88ae1f16a3e58580396a5d31d4d77210.jpg
./super-resolution-rdn/person/images-of-pistol-in-hand_24_jpeg_jpg.rf.263936679204d70ccd00e13f3111be23.jpg
./super-resolution-rdn/person/mgd_custom_frame1737_jpg.rf.6e82171efd22b487c6eb01cdfa401d23.jpg
./super-resolution-rdn/person/mgd_custom_frame164_jpg.rf.13ae32f9111f7b87ddc76398eb6b945c.jpg
./super-resolution-rdn/person/gmd_4_frame67_jpg.rf.ad437c5e6ffe0fc9fd99db751f722290.jpg
./super-resolution-rdn/person/gmd_4_frame47_jpg.rf.8a10f48f58900ab92d3d55bfb63c08dc.jpg
./super-resolution-rdn/person/images-of-

## Cargar en drive modelo

In [ ]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

Mounted at /content/drive/


In [ ]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./super-resolution-esrgan-personas-armadas.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/personas-armadas-detectadas

In [ ]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./super-resolution-rdn.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/personas-armadas-detectadas